In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch
from typing import List
from tqdm import tqdm

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
expla_df = pd.read_csv("../data/explagraphs/train_v3.tsv", sep="\t")
copa_df = pd.read_csv("../data/copa/train_v3.tsv", sep="\t")

In [ ]:
columns = {
    'linked_paths': 'linked_paths',
    'gold_graph': 'gold_graph',
    'generated_graph_linked': 'generated_graph_linked',
    'generated_graph_gold': 'generated_graph_gold',
    'retrieved_graph': 'retrieved_graph'
}

In [ ]:
def string_stripper(s: str):
    return s.strip().replace('[', '').replace(']', '').replace(',', '').replace('_', '').replace('\'', '')

In [ ]:
def get_avg_similarity(df, is_expla=False):
    stats = {}
    for key, value in columns.items():
        l = df[value].to_numpy().tolist()
        res = []
        for idx, e in enumerate(tqdm(l)):
            try:
                e = string_stripper(e)
                if is_expla:
                    context = df.iloc[idx]["belief"] + " " + df.iloc[idx]["argument"]
                else:
                    context= df.iloc[idx]["p"] + " " + df.iloc[idx]["a1"] + " " + df.iloc[idx]["a2"]
            except:
                continue
            c = model.encode(context, convert_to_tensor=True, show_progress_bar=False)
            g = model.encode(e, convert_to_tensor=True, show_progress_bar=False)
            res.append(util.cos_sim(c,g).item())            
            
        stats[value] = np.mean(res)            
        
    return stats
        

In [ ]:
a = get_avg_similarity(expla_df, is_expla=True)

In [ ]:
a

In [ ]:
b = get_avg_similarity(copa_df, is_expla=False)

In [ ]:
b